# Instalação e importação das bibliotecas

In [1]:
!pip install numpy pandas matplotlib scikit-learn opencv-python tensorflow keras opencv-python-headless

In [8]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Apenas um aviso informando que não foram encontrados drivers cuda e por conta disso será usada a CPU da máquina ao invés da GPU

# Coleta e processamento de dados

In [3]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized

image = preprocess_image('data_set/000022.jpg')

[ WARN:0@7.008] global loadsave.cpp:241 findDecoder imread_('data_set/000022.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


### Criando CNN (rede neural convolucional)
#### Essa função cria um modelo CNN que passa por várias camadas convolucionais para extrair características das imagens, nas quais foram utilizados a função ReLU (função de ativação que permite que a rede neural aprenda padrões complexos) e o Pooling (operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes) seguida por uma camada densa para classificação final em 3 classes, com a função de ativação softmax com 3 neurônios que transforma a saída em uma distribuição de probabilidades (somando 1), apropriada para tarefas de classificação multiclasses (neste caso, classificando em 3 classes possíveis).

In [9]:
def create_cnn(input_shape):
    model = models.Sequential()

    model.add(layers.Input(shape=input_shape)) # Definie a forma de entrada

    # Camada convolucional + ReLU + Pooling
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())  # Achata a saída para alimentar a camada densa
    model.add(layers.Dense(128, activation='relu'))  # Camada densa
    model.add(layers.Dense(3, activation='softmax'))

    return model

# Definindo a forma de entrada da imagem (altura, largura, canais)
input_shape = (224, 224, 1)  # 1 canal para imagens em escala de cinza
cnn_model = create_cnn(input_shape)

# Compilando o modelo
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Treinamento do modelo

In [10]:
# Gerador de dados com aumentação de imagem
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.5
)

# Diretório das imagens de treino e validação
train_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, 
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, # mudar para 8 quando tiver dataset completo
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='validation'
)

# Treinamento do modelo
history = cnn_model.fit(train_generator, validation_data=validation_generator, epochs=30)

Found 3 images belonging to 3 classes.
Found 3 images belonging to 3 classes.
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - accuracy: 0.5417 - loss: 193.7515 - val_accuracy: 0.3333 - val_loss: 69.9269
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step - accuracy: 0.0000e+00 - loss: 121.5029 - val_accuracy: 0.3333 - val_loss: 10.6832
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - accuracy: 0.5417 - loss: 8.3481 - val_accuracy: 0.3333 - val_loss: 7.1125
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 317ms/step - accuracy: 0.2917 - loss: 6.0415 - val_accuracy: 0.0000e+00 - val_loss: 1.1928
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step - accuracy: 0.8333 - loss: 1.0888 - val_accuracy: 0.3333 - val_loss: 4.5737
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 314ms/step - accuracy: 0.8333 - loss: 1.2305 - val_accuracy: 0.3333 - val_loss: 2.6399
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 313ms/step - accuracy: 0.5417 - loss: 1.9705 - val_accuracy: 0.3333 - val_loss: 1.0691
Epoch 8/30
3/3 ━━━━━━━━━━━

Testes de validação

In [11]:
# Avaliando o modelo no conjunto de validação
val_loss, val_acc = cnn_model.evaluate(validation_generator)
print(f"Validação - Loss: {val_loss}, Acurácia: {val_acc}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.0000e+00 - loss: 1.5140
Validação - Loss: 1.40328848361969, Acurácia: 0.0


Função de predição

In [12]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Adiciona uma dimensão para batch
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)  # Retorna o índice da classe com maior probabilidade
    return predicted_class, prediction

predicted_class, result = predict_image('./test-data/001.jpg', cnn_model)
print(f'Predição: {result}')
print(f'Classe prevista: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predição: [[0.22090256 0.29052022 0.48857716]]
Classe prevista: 2
